# Interactive Map of South Africa's Provinces and Districts (with Esri Satellite)

This notebook shows how to build an interactive web map of South Africa using `Python`, `GeoPandas`, and `Folium`.

The goals are:

- to load administrative boundaries (provinces and districts) from GeoJSON files,
- to display them on top of an **Esri satellite (World Imagery)** basemap,
- and to add simple tooltips so we can see the names of provinces and districts.

## Data sources and links

Below are example sources you can use for this section.  
Before downloading any dataset, always read the data use policy and licence.

- **Administrative boundaries (country + provinces)**
  - SimpleMaps – free South Africa admin boundaries (Shapefile / GeoJSON)  
    - https://simplemaps.com/gis/country/za  
    - Licence: free for personal and commercial use, attribution appreciated.

- **District boundaries** 
  - HDX COD-AB (Common Operational Datasets – Administrative Boundaries) for South Africa  
    - https://data.humdata.org (search for “South Africa administrative level 0–4 boundaries”)

[Resources](https://github.com/Phemelo-R/Les-Hyperion/tree/main/resources/01_introduction) used in this notebook include:

- `resources/01_introduction/south_africa_provinces.geojson`
- `resources/01_introduction/south_africa_districts.geojson`

You can rename these files or paths to match your own folder structure.

## Import Python libraries

In this step we import all the Python libraries we need:

- `pathlib.Path` helps us work with folders and file paths in a clean way.
- `geopandas` can read and work with spatial data such as GeoJSON and shapefiles.
- `folium` builds interactive web maps (it uses JavaScript under the hood, but we do not need to write JavaScript ourselves).
- `GeoJsonTooltip` from folium adds labels that pop up when we hover the mouse over features.
- `IPython.display.IFrame` lets us display an external HTML file *inside* the notebook, like a small web page window.

If you do not have these packages installed, you can install them but uncommenting:

In [2]:
# %pip install geopandas folium

In [3]:
# Import python libraries
from pathlib import Path

import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
from IPython.display import IFrame

## Set up the data folder and file paths

We will assume that our GeoJSON files are stored in a folder called `resources/01_introduction` inside the project directory.

We create a `Path` object for this folder and then create two file paths:

- one for the provinces GeoJSON,
- one for the districts GeoJSON.

> **Note:** This cell does *not* download any data. It just tells Python where to find the files you already have.

In [4]:
# Define the data folder
data_folder = Path("resources/01_introduction")

# Paths to GeoJSON files (these files must already exist in this folder)
provinces_path = data_folder / "south_africa_provinces.geojson"
districts_path = data_folder / "south_africa_districts.geojson"

provinces_path, districts_path

(WindowsPath('resources/01_introduction/south_africa_provinces.geojson'),
 WindowsPath('resources/01_introduction/south_africa_districts.geojson'))

## Load the GeoJSON data with GeoPandas

Here we read the two GeoJSON files into GeoPandas `GeoDataFrame` objects.

- A **GeoDataFrame** is like a normal data table (DataFrame), but it also stores a special `geometry` column that contains shapes (points, lines, polygons).
- Each row is one feature (for example, one province or one district).

We then print:

- how many provinces and districts we have,
- which columns are available in each dataset.

In [5]:
# Read provinces and districts into GeoDataFrames
provinces = gpd.read_file(provinces_path)
districts = gpd.read_file(districts_path)

print("Number of provinces:", len(provinces))
print("Province columns:", provinces.columns.tolist())

print("Number of districts:", len(districts))
print("District columns:", districts.columns.tolist())

Number of provinces: 9
Province columns: ['source', 'id', 'name', 'geometry']
Number of districts: 52
District columns: ['Shape_Area', 'ADM2_EN', 'ADM1_EN', 'geometry']


## Keep only the columns we need and rename them

To keep our map light and simple, we keep only:

- the province name and geometry for the provinces,
- the district name, province name, and geometry for the districts.

We then rename the columns so they have clear, consistent names:

- `name` → `province_name` (provinces)
- `ADM2_EN` → `district_name` (districts)
- `ADM1_EN` → `province_name` (districts)

This makes it easier to write tooltip code later, because we know exactly which column names to use.

In [6]:
# Provinces: keep only name and geometry, and rename the column
provinces = (
    provinces[["name", "geometry"]]
    .rename(columns={"name": "province_name"})
)

# Districts: keep only district name, province name, and geometry, then rename
districts = (
    districts[["ADM2_EN", "ADM1_EN", "geometry"]]
    .rename(columns={"ADM2_EN": "district_name", "ADM1_EN": "province_name"})
)

provinces.head(), districts.head()

(   province_name                                           geometry
 0  Northern Cape  POLYGON ((22.63695 -26.11428, 22.70516 -26.129...
 1  KwaZulu-Natal  POLYGON ((30.19597 -31.07789, 30.1925 -31.0729...
 2     Free State  POLYGON ((25.46747 -30.61312, 25.43161 -30.592...
 3   Eastern Cape  POLYGON ((24.14703 -31.78989, 24.15432 -31.758...
 4        Limpopo  POLYGON ((31.85649 -23.96296, 31.85623 -23.963...,
   district_name  province_name  \
 0    Alfred Nzo   Eastern Cape   
 1       Amajuba  KwaZulu-Natal   
 2      Amathole   Eastern Cape   
 3      Bojanala     North West   
 4  Buffalo City   Eastern Cape   
 
                                             geometry  
 0  MULTIPOLYGON (((29.02187 -30.00443, 29.02401 -...  
 1  MULTIPOLYGON (((30.39753 -27.27376, 30.39762 -...  
 2  MULTIPOLYGON (((28.26665 -31.80154, 28.26809 -...  
 3  MULTIPOLYGON (((27.0399 -24.87201, 27.04122 -2...  
 4  MULTIPOLYGON (((27.35458 -32.67444, 27.37057 -...  )

## Coordinate Reference Systems (CRS) and simplifying shapes

When we draw maps, coordinates must follow a **Coordinate Reference System (CRS)**.

- For web maps (like Folium and most online mapping tools), we normally use **EPSG:4326** (latitude and longitude in degrees).
- For operations that involve distances and shapes (like simplifying geometries in metres), we often switch to a projected CRS like **EPSG:3857** (Web Mercator), where units are approximately metres.

### Why simplify geometries?

If the boundary outlines are very detailed, each polygon contains many points. This makes the map heavier and the notebook bigger. By simplifying geometries:

- we reduce the number of points in each polygon,
- the map loads faster,
- but the overall shape is still recognisable at national scale.

We will:

1. Convert the data to EPSG:3857,
2. Simplify the geometries with a tolerance in metres,
3. Convert back to EPSG:4326 for the web map.

In [7]:
# Project to Web Mercator (EPSG:3857) to simplify in metres
provinces_proj = provinces.to_crs(epsg=3857)
districts_proj = districts.to_crs(epsg=3857)

# Simplify geometries (tolerance is in metres)
provinces_proj["geometry"] = provinces_proj.geometry.simplify(tolerance=5000)
districts_proj["geometry"] = districts_proj.geometry.simplify(tolerance=3000)

# Convert back to WGS84 (EPSG:4326) for folium
provinces_wgs84 = provinces_proj.to_crs(epsg=4326)
districts_wgs84 = districts_proj.to_crs(epsg=4326)

provinces_wgs84.crs, districts_wgs84.crs

(<Geographic 2D CRS: EPSG:4326>
 Name: WGS 84
 Axis Info [ellipsoidal]:
 - Lat[north]: Geodetic latitude (degree)
 - Lon[east]: Geodetic longitude (degree)
 Area of Use:
 - name: World.
 - bounds: (-180.0, -90.0, 180.0, 90.0)
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich,
 <Geographic 2D CRS: EPSG:4326>
 Name: WGS 84
 Axis Info [ellipsoidal]:
 - Lat[north]: Geodetic latitude (degree)
 - Lon[east]: Geodetic longitude (degree)
 Area of Use:
 - name: World.
 - bounds: (-180.0, -90.0, 180.0, 90.0)
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich)

## Find a good map centre

Folium needs a starting centre for the map, given as `[latitude, longitude]`.

To find a natural centre for South Africa, we can:

- take the union of all province geometries,
- compute its centroid (middle point).

We then extract `latitude` and `longitude` from that centroid.

In [8]:
# Calculate a central point for South Africa using the provinces
centroid = provinces_wgs84.unary_union.centroid
center_lat, center_lon = centroid.y, centroid.x

center_lat, center_lon

C:\Users\pheme\AppData\Local\Temp\ipykernel_3512\3736493259.py:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = provinces_wgs84.unary_union.centroid


(-29.000745987165885, 25.086649154826)

## Create a Folium map with an Esri satellite basemap

Now we build the interactive map with Folium.

Instead of the default basemap, we use **Esri World Imagery** (satellite imagery). This is done using an **XYZ tile layer**, where:

- `{z}` is the zoom level,
- `{x}` and `{y}` are tile coordinates.

We set:

- the map centre to the centroid we computed,
- a starting zoom level of `5` to show the whole country,
- `tiles=None` so we can add our own basemap manually.

In [9]:
# Define the Esri World Imagery XYZ tile URL
ESRI_WORLD_IMAGERY = (
    "https://services.arcgisonline.com/ArcGIS/rest/services/"
    "World_Imagery/MapServer/tile/{z}/{y}/{x}"
)

# Create the folium map
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=5,
    tiles=None  # we will add the Esri satellite layer ourselves
)

# Add the Esri satellite tile layer
folium.TileLayer(
    tiles=ESRI_WORLD_IMAGERY,
    attr="Esri World Imagery",
    name="Esri Satellite",
    overlay=False,
    control=True,
).add_to(m)

m

## Add provinces and districts as layers

We now add two layers on top of the satellite image:

1. **Provinces** layer
2. **Districts** layer

For each layer we:

- define a simple style function for line colour, fill colour, and transparency,
- create a `GeoJsonTooltip` so that when we hover over a feature, its name appears.

We also add a `LayerControl` so we can switch layers on and off on the map.

In [10]:
# Style functions for provinces and districts
def province_style(_feature):
    return {
        "fillColor": "#3182bd",
        "color": "#08519c",
        "weight": 1,
        "fillOpacity": 0.25,
    }


def district_style(_feature):
    return {
        "fillColor": "#31a354",
        "color": "#006d2c",
        "weight": 0.7,
        "fillOpacity": 0.15,
    }


# Tooltips for provinces and districts
province_tooltip = GeoJsonTooltip(
    fields=["province_name"],
    aliases=["Province:"],
    sticky=False,
)

district_tooltip = GeoJsonTooltip(
    fields=["district_name", "province_name"],
    aliases=["District:", "Province:"],
    sticky=False,
)

# Add provinces GeoJSON layer
folium.GeoJson(
    provinces_wgs84,
    style_function=province_style,
    tooltip=province_tooltip,
    name="Provinces",
).add_to(m)

# Add districts GeoJSON layer
folium.GeoJson(
    districts_wgs84,
    style_function=district_style,
    tooltip=district_tooltip,
    name="Districts",
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

m

***This brings and end to our introductory section of GIS with python.***